In [1]:
import nltk
from nltk.tokenize import sent_tokenize #make sure to install the corpus
from difflib import SequenceMatcher

'''
Compares `file1` and `file2` for similarity. Set the `threshold`
value to the level of similarity you want to detect (0.5 is a good start).
Then set the `maxLen` variable to around 30 to avoid very short sentences.
This verision supports txt and docx files, use the corresponding two 
functions, or write your own ones for other file formats. 
Note: This algorithm is not very efficient on large files.

Author: christopher.kullenberg@gmail.com

Future work: Make graph of lots of texts that creates edges between 
texts that share similar fragments. 
'''

# Settings:
threshold = 0.5
maxLen = 30
file1 = 'tmp/exempel1.txt'
file2 = 'tmp/exempel2.txt'

def txtParser(fn):
    '''Input: txt-file
    Output: text as string'''
    try:
        thefile = open(fn, 'r', encoding="utf-8")
        thetext = thefile.read()
    except UnicodeError:
        print("Unicode error. Save as Unicode / UTF-8.")
    return(thetext)

def docxParser(fn):
    from docx import Document
    '''Input: docx-file
    Output: text as string
    Note: Build docx from source 
    https://github.com/python-openxml/python-docx'''
    text = ""
    documenttext = Document(fn)
    for d in documenttext.paragraphs:
        text += d.text
    return(text)

In [14]:
def compareSent(document1, document2, threshold, maxLen):
    '''
    Input: Two texts as strings, threshold value as float, maxLen as int.  
    Output: Prints sentences that are more similar than the threshold value for m.ratio().
    Filter: Excludes sentences shorter than maxLen chars. (Recommended: 30)
    '''
    sentences1 = sent_tokenize(document1)
    sentences2 = sent_tokenize(document2)
    print(len(sentences1), len(sentences2))
    for s in sentences1:
        if len(s) > maxLen: 
            for x in sentences2:
                if len(x) > maxLen:
                    m = SequenceMatcher(None, s, x)
                    if m.ratio() > threshold:
                        print("*" * 10)
                        print("Similarity ratio: " + str(round(m.ratio(), 2)))
                        print("\nSentence in File1:\n\n\t" + s)
                        print("\nSentence in File2:\n\n\t" + x)

In [15]:
%%time
compareSent(txtParser(file1), txtParser(file2), threshold, maxLen)

1174 11
**********
Similarity ratio: 1.0

Sentence in File1:

	Du passerar med största sannolikhet en anslagstavla varje dag på väg till skolan eller jobbet.

Sentence in File2:

	Du passerar med största sannolikhet en anslagstavla varje dag på väg till skolan eller jobbet.
**********
Similarity ratio: 0.54

Sentence in File1:

	Du passerar med största sannolikhet en anslagstavla varje dag på väg till skolan eller jobbet.

Sentence in File2:

	Idag ser man vissa likheter mellan en anslagstavla och sociala medier.
**********
Similarity ratio: 1.0

Sentence in File1:

	Vad är det för information som hänger där, på fastnålade papperslappar?

Sentence in File2:

	Vad är det för information som hänger där, på fastnålade papperslappar?
**********
Similarity ratio: 1.0

Sentence in File1:

	Om detta vet forskningen ganska lite eftersom nästan ingen har undersökt anslagstavlorna på ett vetenskapligt sätt.

Sentence in File2:

	Om detta vet forskningen ganska lite eftersom nästan ingen har unde

In [4]:
import concurrent.futures

In [11]:
def match(s, sentences2, threshold, maxLen):
    if len(s) > maxLen: 
        for x in sentences2:
            if len(x) > maxLen:
                m = SequenceMatcher(None, s, x)
                if m.ratio() > threshold:
                    print("*" * 10)
                    print("Similarity ratio: " + str(round(m.ratio(), 2)))
                    print("\nSentence in File1:\n\n\t" + s)
                    print("\nSentence in File2:\n\n\t" + x)
                    return s, x


def comparesentconcurrent(document1, document2, threshold, maxLen):
    '''
    Input: Two texts as strings, threshold value as float, maxLen as int.  
    Output: Prints sentences that are more similar than the threshold value for m.ratio().
    Filter: Excludes sentences shorter than maxLen chars. (Recommended: 30)
    '''
    sentences1 = sent_tokenize(document1)
    sentences2 = sent_tokenize(document2)
    print(len(sentences1))

    with concurrent.futures.ThreadPoolExecutor(max_workers=5) as executor:
        result = {executor.submit(match, s, sentences2, threshold, maxLen): s for s in sentences1} #det är här jag måste lägga loopen
        for future in concurrent.futures.as_completed(result):
            data = future.result()
            #print(amatch)
            if data:
                print(data)
                  
                  
      

In [13]:
%%time
comparesentconcurrent(txtParser(file1), txtParser(file2), threshold, maxLen)

1174
**********
**********Similarity ratio: 1.0


Sentence in File1:

	Du passerar med största sannolikhet en anslagstavla varje dag på väg till skolan eller jobbet.

Sentence in File2:

	Du passerar med största sannolikhet en anslagstavla varje dag på väg till skolan eller jobbet.**********

**********Similarity ratio: 1.0Similarity ratio: 1.0**********

**********

Similarity ratio: 1.0
Sentence in File1:

	Vad är det för information som hänger där, på fastnålade papperslappar?

Sentence in File1:

	Om detta vet forskningen ganska lite eftersom nästan ingen har undersökt anslagstavlorna på ett vetenskapligt sätt.Similarity ratio: 1.0

Similarity ratio: 1.0


Sentence in File1:

	På 80-talet, precis innan internet slog igenom, beskrev man de tidiga datoriserade tjänsterna som en anslagstavla.
Sentence in File2:

	Vad är det för information som hänger där, på fastnålade papperslappar?

Sentence in File2:

	Om detta vet forskningen ganska lite eftersom nästan ingen har undersökt anslags